In [6]:
import numpy as np
import pandas as pd
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.integrate import quad as quad
import emcee

# Load the fits file
data = Table.read('StellarMassesv19.fits', format='fits')

# Filter the data
filter_data = data[(data['logmstar'] + np.log10(data['fluxscale']) > 9) & (data['Z'] > 0.035) & (data['Z'] < 0.065) & (data['fluxscale'] > 0.3) & (data['fluxscale'] < 3)]

fluxscale = filter_data['fluxscale']
z = filter_data['Z']
logM = filter_data['logmstar'] + np.log10(fluxscale)
dlogM = filter_data['dellogmstar']

# Create mass bins
bin_edges = np.arange(9, 12, 0.2)  # You can adjust the bin size
bin_indices = np.digitize(logM, bins=bin_edges)
binned_logM = [logM[bin_indices == i] for i in range(1, len(bin_edges))]

# Get the bin centers
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
bin_counts = np.array([len(binned_logM[i]) for i in range(len(binned_logM))])


C:\Users\catch\AppData\Local\Temp\ipykernel_33420\2633070723.py:12: RuntimeWarning: invalid value encountered in log10
  filter_data = data[(data['logmstar'] + np.log10(data['fluxscale']) > 9) & (data['Z'] > 0.035) & (data['Z'] < 0.065) & (data['fluxscale'] > 0.3) & (data['fluxscale'] < 3)]


In [9]:
from sympy import uppergamma, N

def comoving_volume(z_initial, z_final, H0=70, Omega_M=0.3, Omega_Lambda=0.7):
    c = 3e5 #light speed in km/s as H0 is 70km/s/MPc
    # For a flat universe, Omega_k = 0
    Omega_k = 1 - Omega_M - Omega_Lambda  
    # Hubble distance in Mpc
    DH = c / H0  

    # Angular size distance
    def E(z, Omega_M=Omega_M, Omega_Lambda=Omega_Lambda, Omega_k=Omega_k):
        return np.sqrt(Omega_M * (1 + z)**3 + Omega_k * (1 + z)**2 + Omega_Lambda)

    # Integral for comoving distance DC
    def comoving_distance(z):
        return quad(lambda z: 1/E(z), 0, z)[0] * DH

    # Calculate DC for initial and final redshifts
    DC_initial = comoving_distance(z_initial)
    DC_final = comoving_distance(z_final)

    # Volume of the light cone V_C using the formula for Omega_k = 0 (flat universe)
    VC_initial = (4 * np.pi / 3) * DC_initial**3
    VC_final = (4 * np.pi / 3) * DC_final**3

    # The volume covered between z_initial and z_final is the difference
    V_360 = VC_final - VC_initial
    
    #full sky has 41253 sq degrees and GAMA has 250sq degrees
    V_gama = 143/41253 * V_360
    
    return V_gama

In [ ]:
def lnlike_2model_binned(parvals, bin_centers, bin_counts, return_model=False):
    alpha1, alpha2, log_m_star, log_phi_star1, log_phi_star2 = parvals

    if log_m_star < 8 or log_m_star > 12 or alpha1 >= 2 or alpha2 >= 2 or log_phi_star1 <= -5 or log_phi_star2 <= -5:
        return -np.inf, 0

    def new_sch(log_mass):
        x = 10**(log_mass - log_m_star)
        sch1 = (10**log_phi_star1) * (x**(alpha1 + 1))
        sch2 = (10**log_phi_star2) * (x**(alpha2 + 1))
        return (sch1 + sch2) * np.exp(-x) * np.log(10)

    def new_int():
        [a, b] = [min(bin_centers) - log_m_star, max(bin_centers) - log_m_star]

        gamma1_a = float(N(uppergamma(alpha1 + 1, 10**a)))
        gamma1_b = float(N(uppergamma(alpha1 + 1, 10**b)))
        gamma2_a = float(N(uppergamma(alpha2 + 1, 10**a)))
        gamma2_b = float(N(uppergamma(alpha2 + 1, 10**b)))

        norm1 = (10**log_phi_star1) * (gamma1_a - gamma1_b)
        norm2 = (10**log_phi_star2) * (gamma2_a - gamma2_b)
        return norm1 + norm2

    def Nexp():
        V = comoving_volume(0.035, 0.065)
        phi_int = new_int()
        return V * phi_int
    
    def pslike():
        #expected Number Counts

        #log Likelihood
        Nobs = np.sum(bin_counts)
        Nexp1 = Nexp()
        logL =(-1./2.)*((Nobs-Nexp1)**2)/Nexp1 -(1./2.*np.log(2*np.pi*Nexp1))

        #print(logL, Nobs, Nexp1)
        return logL
    
    
    lnprior_alpha1 = -0.5 * ((-1 - alpha1) / 1.5)**2
    lnprior_alpha2 = -0.5 * ((-1 - alpha2) / 1.5)**2
    lnprior = lnprior_alpha1 + lnprior_alpha2

    sch = new_sch(bin_centers)
    norm = new_int()

    Nexp1 = Nexp()

    if norm == 0:
        return -np.inf, 0

    like = np.clip( sch / norm, 1e-12, np.inf )
    L = np.nansum( np.log( like ) )

    P = pslike()
    
    if np.isnan(L) or np.isnan(P) or np.isinf(L) or np.isinf(P):
        #print(parvals, L, P)
        return -np.inf, 0

    if return_model:
        return like

    return L + lnprior + P, Nexp1


In [8]:
import emcee

# Define the initial positions of the walkers
ndim = 5
nwalkers = 20
p0 = np.random.rand(nwalkers, ndim)
mstar00 = 10.5 + 0.5 * np.random.randn(nwalkers)
alpha00 = -1.5 + 0.3 * np.random.rand(nwalkers)
alpha01 = -0.5 + 0.3 * np.random.rand(nwalkers)
log_phistar00 = np.log10(0.8e-3 + 0.0003 * np.random.rand(nwalkers))
log_phistar01 = np.log10(3e-2 + 0.003 * np.random.rand(nwalkers))

p0 = np.vstack([alpha00, alpha01, mstar00, log_phistar00, log_phistar01]).T

# Initialize the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnlike_2model_binned, args=[bin_centers, bin_counts])

# Run the sampler
state = sampler.run_mcmc(p0, 10000, progress=True)


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (10, 1) + inhomogeneous part.